<a href="https://colab.research.google.com/github/MatCat776/InterviewBot/blob/main/interview_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My Job Interview Bot


## Configuration

### Install Libraries

In [ ]:
%pip install --upgrade --quiet google-genai
%pip install pdfplumber

In [ ]:
import base64

### Configure Gemini

In [ ]:
from google import genai
from google.genai import types
from google.colab import userdata

# create a google api key and save it as a secret in colab before running next
# line
GOOGLE_API_KEY = userdata.get('AI_STUDIO_API_KEY')
model_name = "gemini-2.5-flash-lite"


### Configure google drive

In [ ]:
from google.colab import drive
# next line requires granting permissions to access drive
drive.mount('/content/drive')

### Configure for multi-turn chat

In [ ]:
from google.generativeai import configure
from google.generativeai import GenerativeModel
from google.generativeai.types import HarmCategory, HarmBlockThreshold # Using the type aliases as available

configure(api_key=GOOGLE_API_KEY)


## Obtain source data


### Obtain job posting, txt format

In [ ]:
# Read the content of the text file
with open("/content/drive/My Drive/job_posting.txt", "r") as f:
    job_posting_content = f.read()

### Obtain resume, pdf format
Pulling content from google drive and github doesn't seem to work in colab the same way it does in Virtex AI. For some reason, I have to convert the pdf to txt, then I can use it here.

In [ ]:
import pdfplumber

def convert_pdf_to_txt_pdfplumber(pdf_path, txt_path):
    """Converts a PDF file to a plain text file using pdfplumber."""
    try:
        with pdfplumber.open(pdf_path) as pdf, open(txt_path, "w", encoding="utf-8") as f:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    f.write(text + '\n')  # Add a newline to separate pages
        print(f"Text successfully extracted from {pdf_path} to {txt_path}")
    except Exception as e:
        print(f"Error converting PDF: {e}")

# Example usage:
# text_resume_content = get_text_from_pdf("/content/drive/My Drive/My_Resume.pdf")

def get_text_from_pdf(pdf_path):
  # google drive used as temp storage location
  convert_pdf_to_txt_pdfplumber(pdf_path, "/content/drive/My Drive/temp.txt")
  with open("/content/drive/My Drive/temp.txt", "r") as f:
    text_resume_content = f.read()
    return text_resume_content

In [ ]:
# Get resume text
text_resume_content = get_text_from_pdf("/content/drive/My Drive/My_Resume.pdf")


## Static model settings

In [ ]:
#System instructions
my_instructions = """
You are a job applicant in a job interview. This is your resume:
""" + text_resume_content + """ This is the job posting: """ + job_posting_content

# Safety settings
safety_settings_multiturn = [
    {
        "category": HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        "threshold": HarmBlockThreshold.BLOCK_LOW_AND_ABOVE, # Block low and above probability derogatory content
    },
    {
        "category": HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        "threshold": HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, # Block medium and above probability violent content
    },
    {
        "category": HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        "threshold": HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, # Block medium and above probability violent content
    },
    {
        "category": HarmCategory.HARM_CATEGORY_HARASSMENT,
        "threshold": HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, # Block medium and above probability violent content
    },
]


## Start up model

In [ ]:
model = GenerativeModel(model_name=model_name, system_instruction=my_instructions, safety_settings=safety_settings_multiturn)

chat = model.start_chat(history=[])

# function to display question and answer
def ask_question(question):
  user_message = question
  response = chat.send_message(user_message)
  print(f"User: {user_message}")
  print(f"Gemini: {response.text}")

## Interview Questions

In [ ]:
ask_question("What is your name?")
ask_question("Why are you a good fit for this job?")
ask_question("What is your salary expectation?")